# Generating 4 classes

In [1]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import cv2
import PIL.Image
import torch
import legacy
import dnnlib
import numpy as np
import tqdm.notebook as tqdm
import imageio
import shutil
import os
from PIL import Image
import gradio as gr
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Path to the trained model
model_path = os.path.join(os.getcwd(), "models", "oct.pkl") # ["NORMAL", "CNV", "DRUSEN", "DME"]
model_path = os.path.join(os.getcwd(), "models", "oct_256_resized_1600.pkl") # ["DRUSEN", "NORMAL", "DME", "CNV"]

# Directory where temporary files should be stored
directory = os.path.join(os.getcwd(), "demo")

# Available classes to choose from
classes = ["DRUSEN", "NORMAL", "DME", "CNV"]
classes_dict = {"DRUSEN": 0, "NORMAL": 1, "DME": 2, "CNV": 3}

# Torch device
device = torch.device('cuda')

# Generate an image

In [3]:
# Method to generate an image by choosing out of four classes 
def generate_noise_image(choice):
    random_seed = np.random.randint(1, 9999)
    execution = f"python generate.py --outdir={directory} --seeds={random_seed} --network={model_path} --class={classes_dict[choice]} --vector-mode=True"
    os.system(execution)    
    img_path = os.path.join(os.getcwd(), directory, f"{classes_dict[choice]}_{str(random_seed).zfill(4)}.png")
    img = Image.open(img_path)
    return img

iface1 = gr.Interface(
    fn=generate_noise_image,
    inputs=gr.inputs.Dropdown(choices=classes, default="NORMAL", label="Class"),
    outputs=gr.outputs.Image(type="pil").style(height=256, width=256)
)

# Interpolating between x images (z-vectors)

In [4]:
# Method to create a video of an interpolation between x images
def generate_noise_image(sequence, frames, n_steps):
    try:
        shutil.rmtree(directory)
    except OSError as error:
        print(f"Failed to delete directory '{directory}': {error}")
    
    counts = [sequence.count(str(i)) for i in range(4)]
    random_seeds = np.sort(np.random.randint(1, 9999, sum(counts)))
    print(counts)
    print(random_seeds)
    lvecs = []
    
    for i in range(sum(counts)):
        execution = f"python generate.py --outdir={directory} --seeds={random_seeds[i]} --network={model_path} --class={sequence[i]} --vector-mode=True"
        os.system(execution)
        print("Generated image!")
        
        ################################################################################################################
        class_idx = int(sequence[i])

        with dnnlib.util.open_url(model_path) as fp:
            G = legacy.load_network_pkl(fp)['G_ema'].requires_grad_(False).to(device)

        @torch.no_grad()
        def map_z_to_w(z, G):
            label = torch.zeros((1, G.c_dim), device=device)
            label[:, class_idx] = 1
            w = G.mapping(z.to(device), label)
            return w

        # Load z from file.
        z_path = os.path.join(os.getcwd(), "demo", f"{sequence[i]}_{str(random_seeds[i]).zfill(4)}.npy")
        z_np = np.load(z_path)

        # Convert `z_np` to a PyTorch tensor.
        z = torch.as_tensor(z_np, device=device)

        # Convert z to w.
        w = map_z_to_w(z, G)
        ################################################################################################################
        
        lvecs.append(w)

    FPS = 60
    FREEZE_STEPS = 30
    STEPS = int(frames)
    
    with dnnlib.util.open_url(model_path) as fp:
        G = legacy.load_network_pkl(fp)['G_ema'].requires_grad_(False).to(device)  # type: ignore
    
    video = imageio.get_writer(f'{directory}/video.mp4', mode='I', fps=FPS, codec='libx264', bitrate='16M')

    for i in range(sum(counts) - 1):# load z_arr from npz file
        diff = lvecs[i+1] - lvecs[i]
        step = diff / STEPS
        current = lvecs[i].clone()
        target_uint8 = np.array([256, 256, 3], dtype=np.uint8)


        for j in range(STEPS):
            z = current.to(device)
            synth_image = G.synthesis(z, noise_mode='const')
            synth_image = (synth_image + 1) * (255 / 2)
            synth_image = synth_image.permute(0, 2, 3, 1).clamp(0, 255).to(torch.uint8)[0].cpu().numpy()

            repeat = FREEZE_STEPS if j == 0 or j == (STEPS - 1) else 1

            for i in range(repeat):
                video.append_data(synth_image)
            current = current + step

    video.close()
    
    return f"{directory}/video.mp4"

sequence = gr.inputs.Textbox(default="13", label="Sequence input")
n_steps = gr.inputs.Number(default=100, label="Latent Steps")
frames = gr.inputs.Number(default=240, label="Frames")
output = gr.outputs.Video(type="mp4").style(height=256, width=256)

iface2 = gr.Interface(
    fn=generate_noise_image,
    inputs=[sequence, frames, n_steps],
    outputs=output,
    description="This Gradio interface generates a video by morphing in the latent space between given class images. To use it, enter a string consisting only of letters from 0-3 to serve as class labels."
)

# Interpolating between x images (approximation)

In [5]:
# Method to create a video of an interpolation between x images
def generate_noise_image(sequence, frames, n_steps):
    try:
        shutil.rmtree(directory)
    except OSError as error:
        print(f"Failed to delete directory '{directory}': {error}")
    
    counts = [sequence.count(str(i)) for i in range(4)]
    random_seeds = np.sort(np.random.randint(1, 9999, sum(counts)))
    print(counts)
    print(random_seeds)
    lvecs = []
    
    for i in range(sum(counts)):
        execution = f"python generate.py --outdir={directory} --seeds={random_seeds[i]} --network={model_path} --class={sequence[i]}"
        os.system(execution)
        
        outdir_path = os.path.join(directory, "projections", str(i).zfill(2))
        target_dir = os.path.join(directory, f"{sequence[i]}_{str(random_seeds[i]).zfill(4)}.png")
        execution = f"python projector.py --outdir={outdir_path} --target={target_dir} --network={model_path} --num-steps={int(n_steps)} --save-video=True"
        os.system(execution)
        
        lvec_path = os.path.join(os.getcwd(), directory, "projections", f"{str(i).zfill(2)}", "projected_w.npz") 
        lvecs.append(np.load(lvec_path)['w'])

    FPS = 60
    FREEZE_STEPS = 30
    STEPS = int(frames)
    
    with dnnlib.util.open_url(model_path) as fp:
        G = legacy.load_network_pkl(fp)['G_ema'].requires_grad_(False).to(device)  # type: ignore
    
    video_path = os.path.join(os.getcwd(), directory, "video.mp4")
    video = imageio.get_writer(video_path, mode='I', fps=FPS, codec='libx264', bitrate='16M')

    for i in range(sum(counts) - 1):
        diff = lvecs[i+1] - lvecs[i]
        step = diff / STEPS
        current = lvecs[i].copy()
        target_uint8 = np.array([256, 256, 3], dtype=np.uint8)


        for j in range(STEPS):
            z = torch.from_numpy(current).to(device)
            synth_image = G.synthesis(z, noise_mode='const')
            synth_image = (synth_image + 1) * (255 / 2)
            synth_image = synth_image.permute(0, 2, 3, 1).clamp(0, 255).to(torch.uint8)[0].cpu().numpy()

            repeat = FREEZE_STEPS if j == 0 or j == (STEPS - 1) else 1

            for i in range(repeat):
                video.append_data(synth_image)
            current = current + step

    video.close()
    
    return os.path.join(os.getcwd(), directory, "video.mp4")

sequence = gr.inputs.Textbox(default="13", label="Sequence input")
n_steps = gr.inputs.Number(default=100, label="Latent Steps")
frames = gr.inputs.Number(default=240, label="Frames")
output = gr.outputs.Video(type="mp4").style(height=256, width=256)

iface3 = gr.Interface(
    fn=generate_noise_image,
    inputs=[sequence, frames, n_steps],
    outputs=output,
    description="This Gradio interface generates a video by morphing in the latent space between given class images. To use it, enter a string consisting only of letters from 0-3 to serve as class labels."
)

# Generate variations of an image

In [6]:
# Method to create a video of an interpolation between x images
def generate_noise_image(sequence, frames):
    try:
        shutil.rmtree(directory)
    except OSError as error:
        print(f"Failed to delete directory '{directory}': {error}")
    
    counts = [sequence.count(str(i)) for i in range(4)]
    random_seeds = [np.random.randint(1, 9999)] * sum(counts)
    print(counts)
    print(random_seeds)
    lvecs = []
    
    for i in range(sum(counts)):
        execution = f"python generate.py --outdir={directory} --seeds={random_seeds[i]} --network={model_path} --class={sequence[i]} --vector-mode=True"
        os.system(execution)
        print("Generated image!")
        
        ################################################################################################################
        class_idx = int(sequence[i])

        with dnnlib.util.open_url(model_path) as fp:
            G = legacy.load_network_pkl(fp)['G_ema'].requires_grad_(False).to(device)

        @torch.no_grad()
        def map_z_to_w(z, G):
            label = torch.zeros((1, G.c_dim), device=device)
            label[:, class_idx] = 1
            w = G.mapping(z.to(device), label)
            return w

        # Load z from file.
        # z_path = os.path.join(os.getcwd(), "demo", f"seed{str(random_seeds[i]).zfill(4)}.npy")
        z_path = os.path.join(os.getcwd(), "demo", f"{sequence[i]}_{str(random_seeds[i]).zfill(4)}.npy")
        z_np = np.load(z_path)

        # Convert `z_np` to a PyTorch tensor.
        z = torch.as_tensor(z_np, device=device)

        # Convert z to w.
        w = map_z_to_w(z, G)
        ################################################################################################################
        
        lvecs.append(w)

    FPS = 60
    FREEZE_STEPS = 30
    STEPS = int(frames)
    
    with dnnlib.util.open_url(model_path) as fp:
        G = legacy.load_network_pkl(fp)['G_ema'].requires_grad_(False).to(device)  # type: ignore
    
    video = imageio.get_writer(f'{directory}/video.mp4', mode='I', fps=FPS, codec='libx264', bitrate='16M')

    for i in range(sum(counts) - 1):# load z_arr from npz file
        diff = lvecs[i+1] - lvecs[i]
        step = diff / STEPS
        current = lvecs[i].clone()
        target_uint8 = np.array([256, 256, 3], dtype=np.uint8)


        for j in range(STEPS):
            z = current.to(device)
            synth_image = G.synthesis(z, noise_mode='const')
            synth_image = (synth_image + 1) * (255 / 2)
            synth_image = synth_image.permute(0, 2, 3, 1).clamp(0, 255).to(torch.uint8)[0].cpu().numpy()

            repeat = FREEZE_STEPS if j == 0 or j == (STEPS - 1) else 1

            for i in range(repeat):
                video.append_data(synth_image)
            current = current + step

    video.close()
    
    return f"{directory}/video.mp4"

sequence = gr.inputs.Textbox(default="13", label="Sequence input")
frames = gr.inputs.Number(default=240, label="Frames")
output = gr.outputs.Video(type="mp4").style(height=256, width=256)

iface4 = gr.Interface(
    fn=generate_noise_image,
    inputs=[sequence, frames],
    outputs=output,
    description="This Gradio interface generates a video by morphing in the latent space between given class images. To use it, enter a string consisting only of letters from 0-3 to serve as class labels."
)

# Multiple Tabs

In [7]:
gr.TabbedInterface(
    [iface1, iface2, iface3, iface4], ["Generate an image", "Generate a video (original)", "Generate a video (approximated)", "Generate variations of an image"]
).launch(share=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://067b0bbb-ac19-4f48.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


[0, 1, 0, 1]
[1007 3006]
Generated image!
Setting up PyTorch plugin "bias_act_plugin"... [0, 1, 0, 1]
[6079 7666]
Done.
Generated image!
Setting up PyTorch plugin "upfirdn2d_plugin"... Done.
[0, 1, 0, 1]
[1431, 1431]
Generated image!
Generated image!
